In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
#import os
#mysql_jar_path = os.path.abspath("mysql-connector-j-9.0.0.jar")
#print("Caminho completo do JAR:", mysql_jar_path)


In [26]:
import numpy as np
import pandas as pd
import matplotlib
import tensorflow as tf
import sklearn

In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression, RandomForestRegressor
from pyspark.ml.feature import VectorAssembler
import pandas as pd
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import col, sum



In [28]:
pip install yfinance

Note: you may need to restart the kernel to use updated packages.


In [29]:
import yfinance as yf

In [30]:


ticker = "ABNB"
data = yf.download(ticker,
                   start="2018-01-01", 
                   end="2020-12-31")    

print(data.head())



[*********************100%***********************]  1 of 1 completed

Price            Close        High         Low        Open    Volume
Ticker            ABNB        ABNB        ABNB        ABNB      ABNB
Date                                                                
2020-12-10  144.710007  165.000000  141.250000  146.000000  70447500
2020-12-11  139.250000  151.500000  135.100006  146.550003  26980800
2020-12-14  130.000000  135.300003  125.160004  135.000000  16966100
2020-12-15  124.800003  127.599998  121.500000  126.690002  10914400
2020-12-16  137.990005  142.000000  124.910004  125.830002  20409600


In [31]:
csv_ABNB = f"{ticker}_ABNB_historical_train_data_spark.csv"
data.to_csv(csv_ABNB)

In [32]:
df_ABNB= pd.read_csv("ABNB_ABNB_historical_train_data_spark.csv")

In [33]:
df_ABNB.head()

,Price,Close,High,Low,Open,Volume
0,Ticker,ABNB,ABNB,ABNB,ABNB,ABNB
1,Date,NaN,NaN,NaN,NaN,NaN
2,2020-12-10,144.7100067138672,165.0,141.25,146.0,70447500
3,2020-12-11,139.25,151.5,135.10000610351562,146.5500030517578,26980800
4,2020-12-14,130.0,135.3000030517578,125.16000366210938,135.0,16966100


In [34]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit

# Parar sessão anterior, se necessário
try:
    spark.stop()
except:
    pass

# Criar nova SparkSession com o caminho exato do JAR
spark = SparkSession.builder \
    .appName("PySpark CSV to MySQL") \
    .master("local") \
    .config("spark.jars", "/usr/local/spark-3.4.4-bin-hadoop3/jars/mysql-connector-j-8.0.33.jar") \
    .getOrCreate()

# Caminho do CSV
file_path = "file:///home/hduser/novo/ABNB_ABNB_historical_train_data_spark.csv"

# Leitura do CSV
df = spark.read.csv(file_path, header=True, inferSchema=True)

# Visualizar as primeiras 5 linhas
df.show(5)


+----------+-----------------+-----------------+------------------+-----------------+--------+
|     Price|            Close|             High|               Low|             Open|  Volume|
+----------+-----------------+-----------------+------------------+-----------------+--------+
|    Ticker|             ABNB|             ABNB|              ABNB|             ABNB|    ABNB|
|      Date|             null|             null|              null|             null|    null|
|2020-12-10|144.7100067138672|            165.0|            141.25|            146.0|70447500|
|2020-12-11|           139.25|            151.5|135.10000610351562|146.5500030517578|26980800|
|2020-12-14|            130.0|135.3000030517578|125.16000366210938|            135.0|16966100|
+----------+-----------------+-----------------+------------------+-----------------+--------+
only showing top 5 rows



In [35]:
# print the data type info
df.printSchema()

root
 |-- Price: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- Volume: string (nullable = true)



In [36]:
df.show(5)

+----------+-----------------+-----------------+------------------+-----------------+--------+
|     Price|            Close|             High|               Low|             Open|  Volume|
+----------+-----------------+-----------------+------------------+-----------------+--------+
|    Ticker|             ABNB|             ABNB|              ABNB|             ABNB|    ABNB|
|      Date|             null|             null|              null|             null|    null|
|2020-12-10|144.7100067138672|            165.0|            141.25|            146.0|70447500|
|2020-12-11|           139.25|            151.5|135.10000610351562|146.5500030517578|26980800|
|2020-12-14|            130.0|135.3000030517578|125.16000366210938|            135.0|16966100|
+----------+-----------------+-----------------+------------------+-----------------+--------+
only showing top 5 rows



In [37]:
# Remove the first two lines (CRL and null)
df = df.filter(df.Price != "Ticker").filter(df.Price != "Date")

In [38]:
df = df.toDF("Date", "Close", "High", "Low", "Open", "Volume")

In [39]:


df.show(5)



+----------+------------------+-----------------+------------------+------------------+--------+
|      Date|             Close|             High|               Low|              Open|  Volume|
+----------+------------------+-----------------+------------------+------------------+--------+
|2020-12-10| 144.7100067138672|            165.0|            141.25|             146.0|70447500|
|2020-12-11|            139.25|            151.5|135.10000610351562| 146.5500030517578|26980800|
|2020-12-14|             130.0|135.3000030517578|125.16000366210938|             135.0|16966100|
|2020-12-15|124.80000305175781|127.5999984741211|             121.5|126.69000244140625|10914400|
|2020-12-16|137.99000549316406|            142.0|124.91000366210938|125.83000183105469|20409600|
+----------+------------------+-----------------+------------------+------------------+--------+
only showing top 5 rows



In [40]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- Volume: string (nullable = true)



In [41]:


# # Count Number of nulls in all Column
from pyspark.sql.functions import col, sum

df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).show()



+----+-----+----+---+----+------+
|Date|Close|High|Low|Open|Volume|
+----+-----+----+---+----+------+
|   0|    0|   0|  0|   0|     0|
+----+-----+----+---+----+------+



In [42]:
# convert string double
df = df.withColumn("Close", col("Close").cast("double"))

In [43]:
df.printSchema()


root
 |-- Date: string (nullable = true)
 |-- Close: double (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- Volume: string (nullable = true)



In [44]:


df = df.withColumn("High", col("High").cast("double"))
df = df.withColumn("Low", col("Low").cast("double"))
df = df.withColumn("Open", col("Open").cast("double"))
df = df.withColumn("Volume", col("Volume").cast("double"))
df = df.withColumn("Date", col("Date").cast("Date"))



In [45]:


df.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Close: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Open: double (nullable = true)
 |-- Volume: double (nullable = true)



In [46]:
import mysql.connector
print(mysql.connector.__version__)

9.3.0


In [47]:
df.show(5)

+----------+------------------+-----------------+------------------+------------------+---------+
|      Date|             Close|             High|               Low|              Open|   Volume|
+----------+------------------+-----------------+------------------+------------------+---------+
|2020-12-10| 144.7100067138672|            165.0|            141.25|             146.0|7.04475E7|
|2020-12-11|            139.25|            151.5|135.10000610351562| 146.5500030517578|2.69808E7|
|2020-12-14|             130.0|135.3000030517578|125.16000366210938|             135.0|1.69661E7|
|2020-12-15|124.80000305175781|127.5999984741211|             121.5|126.69000244140625|1.09144E7|
|2020-12-16|137.99000549316406|            142.0|124.91000366210938|125.83000183105469|2.04096E7|
+----------+------------------+-----------------+------------------+------------------+---------+
only showing top 5 rows



In [48]:
# # Count Number of nulls in all Column
from pyspark.sql.functions import col, sum

df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).show()

+----+-----+----+---+----+------+
|Date|Close|High|Low|Open|Volume|
+----+-----+----+---+----+------+
|   0|    0|   0|  0|   0|     0|
+----+-----+----+---+----+------+



In [49]:
# MySQL connection properties
url = "jdbc:mysql://localhost:3306/companies"  # MySQL URL
properties = {
    "user": "root",
    "password": "password",
    "driver": "com.mysql.cj.jdbc.Driver"
}

In [50]:
# Leitura do CSV
#df = spark.read.csv(file_path, header=True, inferSchema=True)

# Adiciona a coluna 'nome' com valor fixo "AAPL"
df_comp_nome = df.withColumn("nome", lit("ABNB"))

# Escreve no banco MySQL
df_comp_nome.write \
    .format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/companies") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", "ABNB_historico") \
    .option("user", "root") \
    .option("password", "password") \
    .mode("overwrite") \
    .save()


In [51]:
# # Count Number of nulls in all Column
from pyspark.sql.functions import col, sum

df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).show()

+----+-----+----+---+----+------+
|Date|Close|High|Low|Open|Volume|
+----+-----+----+---+----+------+
|   0|    0|   0|  0|   0|     0|
+----+-----+----+---+----+------+



mySQL

In [1]:
import pandas as pd
import pymysql

conn = pymysql.connect(
    host='localhost',
    user='root',
    password='password',
    database='companies'
)

df = pd.read_sql("SELECT * FROM ABNB_historico", conn)
df.to_csv("ABNB_historico_MYsql.csv", index=False)


/tmp/ipykernel_3631/736191107.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM ABNB_historico", conn)


In [2]:
df_ABNB_Mysql= pd.read_csv("ABNB_historico_MYsql.csv")


In [3]:
df_ABNB_Mysql.head()

,Date,Close,High,Low,Open,Volume,nome
0,2020-12-10,144.710007,165.000000,141.250000,146.000000,70447500.0,ABNB
1,2020-12-11,139.250000,151.500000,135.100006,146.550003,26980800.0,ABNB
2,2020-12-14,130.000000,135.300003,125.160004,135.000000,16966100.0,ABNB
3,2020-12-15,124.800003,127.599998,121.500000,126.690002,10914400.0,ABNB
4,2020-12-16,137.990005,142.000000,124.910004,125.830002,20409600.0,ABNB
